In [1]:
import os
import warnings

import pandas as pd
import numpy as np

from scipy.stats import binned_statistic_2d
from tqdm import tqdm

warnings.filterwarnings('ignore')
PATH_TO_DATA = '../data'

In [2]:
# bottom x left     and    top x right
longitude_bottom = -74.25559
longitude_top = -73.70001

latitude_left = 40.49612
latitude_right = 40.91553

def clean_data(df):
    pred = (df['passenger_count'] != 0) & (df['trip_distance'] != 0) & \
    (df['tpep_pickup_datetime'] != df['tpep_dropoff_datetime'])  & (df['dropoff_longitude'] >= longitude_bottom) & \
    (df['dropoff_longitude'] <= longitude_top)  & (df['dropoff_latitude'] >= latitude_left) & \
    (df['dropoff_latitude'] <= latitude_right)
    
    df = df[pred]
    df['tpep_dropoff_datetime'] = df['tpep_dropoff_datetime'].apply(lambda x: x.replace(minute=0, second=0))
    
    return df

In [9]:
def aggregate_df(df):
    binx=np.sort(regions.west.unique())[1:]
    biny=np.sort(regions.south.unique())[1:]

    x = df.dropoff_longitude.values
    y = df.dropoff_latitude.values

    _, _, _, regions_ids = binned_statistic_2d(x, y, regions, 'count', bins=[binx, biny])

    regions_ids = regions_ids + 1
    
    df['region'] = regions_ids
    
    all_times = df['tpep_dropoff_datetime'].view('int64')

    binx = regions.index.values.tolist() + [2501]
    biny = np.sort(all_times.unique()).tolist() + [np.datetime64('2016-08-01T00:00:00.000000000').view('int64')]

    x = df.region.values
    y = all_times

    matrix, _, _, _ = binned_statistic_2d(x, y, None, 'count', bins=[binx, biny])
    
    aggregated_df = pd.DataFrame(matrix, index=regions.index.values, columns= np.sort(df['tpep_dropoff_datetime'].unique()))
    
    return aggregated_df

In [4]:
regions = pd.read_csv(os.path.join(PATH_TO_DATA, 'regions.csv'), index_col=0, delimiter=';')

In [16]:
dates = ['tpep_pickup_datetime', 'tpep_dropoff_datetime']
columns = ['passenger_count', 'trip_distance', 'dropoff_longitude', 'dropoff_latitude'] + dates
monthes = [1, 2, 3, 4, 6]
for i in tqdm(monthes):
    file = 'yellow_tripdata_2016-0%d.csv' % i
    data = pd.read_csv(os.path.join(PATH_TO_DATA, file), parse_dates=dates, usecols=columns)
    data = clean_data(data)
    data = aggregate_df(data)
    data.to_csv(os.path.join(PATH_TO_DATA, 'aggregated_destination_'+ file))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [02:06<00:00, 126.34s/it]


In [103]:
data = pd.read_csv(os.path.join(PATH_TO_DATA, 'aggregated_destination_yellow_tripdata_2016-01.csv'), index_col=0)

for i in tqdm(range(2, 7)):
    file = 'aggregated_destination_yellow_tripdata_2016-0%d.csv' % i
    df = pd.read_csv(os.path.join(PATH_TO_DATA, file), index_col=0)

    cols = [c for c in df.columns if c not in data.columns]
    data = data.apply(lambda x: x + df[x.name] if x.name in df.columns else x)
    data = pd.concat([data, df[cols]], axis=1)

data.to_csv(os.path.join(PATH_TO_DATA, 'destination.csv'))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:03<00:00,  1.42it/s]


In [96]:
may = pd.read_csv(os.path.join(PATH_TO_DATA, 'aggregated_yellow_tripdata_2016-05.csv'), index_col=0)
means = may.mean(axis=1)
means = means[means >= 5]
means.shape

(102,)

In [97]:
train_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train.csv'), index_col=0, parse_dates=[0])
test_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_05.csv'), index_col=0, parse_dates=[0])
kaggle_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_06.csv'), index_col=0, parse_dates=[0])


concat_df = pd.concat((train_df, test_df, kaggle_df))
print(concat_df.shape)
concat_df.head()

(4368, 102)


,1075,1076,1077,1125,1126,1127,1128,1129,1130,1131,...,1630,1684,1733,1734,1783,2068,2069,2118,2119,2168
2016-01-01 00:00:00,80.0,144.0,50.0,77.0,319.0,402.0,531.0,617.0,846.0,267.0,...,12.0,0.0,2.0,44.0,5.0,41.0,4.0,70.0,7.0,66.0
2016-01-01 01:00:00,91.0,211.0,49.0,134.0,404.0,420.0,370.0,453.0,594.0,224.0,...,29.0,0.0,5.0,2.0,2.0,4.0,0.0,47.0,1.0,29.0
2016-01-01 02:00:00,90.0,146.0,23.0,110.0,393.0,425.0,313.0,366.0,377.0,138.0,...,47.0,0.0,3.0,0.0,4.0,0.0,0.0,69.0,1.0,14.0
2016-01-01 03:00:00,32.0,87.0,16.0,62.0,252.0,399.0,324.0,309.0,327.0,166.0,...,46.0,0.0,2.0,4.0,5.0,1.0,0.0,21.0,0.0,9.0
2016-01-01 04:00:00,24.0,43.0,10.0,53.0,145.0,254.0,264.0,333.0,318.0,145.0,...,43.0,0.0,0.0,1.0,1.0,0.0,0.0,26.0,1.0,6.0


In [105]:
data = data.loc[means.index].T
data.index = pd.to_datetime(data.index)

In [112]:
data.head()

,1075,1076,1077,1125,1126,1127,1128,1129,1130,1131,...,1630,1684,1733,1734,1783,2068,2069,2118,2119,2168
2016-01-01 00:00:00,74.0,154.0,53.0,74.0,204.0,275.0,274.0,306.0,442.0,152.0,...,34.0,0.0,5.0,2.0,3.0,1.0,0.0,3.0,0.0,1.0
2016-01-01 01:00:00,107.0,218.0,66.0,119.0,336.0,282.0,339.0,454.0,563.0,200.0,...,52.0,0.0,16.0,4.0,1.0,4.0,0.0,3.0,0.0,2.0
2016-01-01 02:00:00,93.0,194.0,37.0,103.0,333.0,216.0,260.0,341.0,370.0,125.0,...,59.0,0.0,15.0,3.0,12.0,2.0,0.0,4.0,2.0,1.0
2016-01-01 03:00:00,72.0,183.0,36.0,88.0,278.0,149.0,209.0,240.0,294.0,116.0,...,75.0,0.0,13.0,24.0,10.0,6.0,1.0,10.0,3.0,9.0
2016-01-01 04:00:00,76.0,127.0,23.0,55.0,171.0,122.0,160.0,197.0,201.0,70.0,...,61.0,0.0,14.0,52.0,17.0,14.0,1.0,15.0,7.0,13.0


In [115]:
indexes = [i in concat_df.index.values for i in data.index.values]
data = data.loc[indexes]

In [117]:
data.shape

(4368, 102)

In [120]:
data = data.sort_index()
data.head()

,1075,1076,1077,1125,1126,1127,1128,1129,1130,1131,...,1630,1684,1733,1734,1783,2068,2069,2118,2119,2168
2016-01-01 00:00:00,74.0,154.0,53.0,74.0,204.0,275.0,274.0,306.0,442.0,152.0,...,34.0,0.0,5.0,2.0,3.0,1.0,0.0,3.0,0.0,1.0
2016-01-01 01:00:00,107.0,218.0,66.0,119.0,336.0,282.0,339.0,454.0,563.0,200.0,...,52.0,0.0,16.0,4.0,1.0,4.0,0.0,3.0,0.0,2.0
2016-01-01 02:00:00,93.0,194.0,37.0,103.0,333.0,216.0,260.0,341.0,370.0,125.0,...,59.0,0.0,15.0,3.0,12.0,2.0,0.0,4.0,2.0,1.0
2016-01-01 03:00:00,72.0,183.0,36.0,88.0,278.0,149.0,209.0,240.0,294.0,116.0,...,75.0,0.0,13.0,24.0,10.0,6.0,1.0,10.0,3.0,9.0
2016-01-01 04:00:00,76.0,127.0,23.0,55.0,171.0,122.0,160.0,197.0,201.0,70.0,...,61.0,0.0,14.0,52.0,17.0,14.0,1.0,15.0,7.0,13.0


In [121]:
data.to_csv(os.path.join(PATH_TO_DATA, 'small_destination.csv'))